# 玖零猴的Demo之BraTS3d网络预处理

## BraTS2018 训练集

In [18]:
from __future__ import print_function, division
import numpy as np
import SimpleITK as sitk
import os

flair_name = "_flair.nii.gz"
t1_name = "_t1.nii.gz"
t1ce_name = "_t1ce.nii.gz"
t2_name = "_t2.nii.gz"
mask_name = "_seg.nii.gz"

input_bratshgg_path = r"F:\Brats_data\MICCAI_BraTS_2018_Data_Training\HGG"
input_bratslgg_path = r"F:\Brats_data\MICCAI_BraTS_2018_Data_Training\LGG"
input_bratshgg_path2019 = r"F:\Brats_data\MICCAI_BraTS_2019_Data_Training\HGG"
input_bratslgg_path2019 = r"F:\Brats_data\MICCAI_BraTS_2019_Data_Training\LGG"

output_trainImage = r"F:\Brats_data\process_data\trainImage"#r"D:\Project\CollegeDesign\dataset\BraTs3D\trainImage"
output_trainMask = r"F:\Brats_data\process_data\trainMask"#r"D:\Project\CollegeDesign\dataset\BraTs3D\trainMask"
output_testImage = r"F:\Brats_data\process_data\testImage"
output_testMask = r"F:\Brats_data\process_data\testMask"

BLOCKSIZE = (16, 160, 160) #每个分块的大小

In [19]:
bratshgg_path = input_bratshgg_path
bratslgg_path = input_bratslgg_path
if not os.path.exists(bratshgg_path):
    print("bratshgg_path 不存在")
if not os.path.exists(bratslgg_path):
    print("bratshgg_path 不存在")

In [20]:
bratshgg_path2 = input_bratshgg_path2019
bratslgg_path2 = input_bratslgg_path2019
if not os.path.exists(bratshgg_path2):
    print("bratshgg_path2019 不存在")
if not os.path.exists(bratslgg_path2):
    print("bratshgg_path2019 不存在")

In [21]:
trainImage = output_trainImage
trainMask = output_trainMask
if not os.path.exists(trainImage):
    os.makedirs(output_trainImage)
    print("trainImage 输出目录创建成功")
if not os.path.exists(trainMask):
    os.makedirs(trainMask)
    print("trainMask 输出目录创建成功")

In [22]:
testImage = output_testImage
testMask = output_testMask
if not os.path.exists(testImage):
    os.makedirs(testImage)
    print("testImage 输出目录创建成功")
if not os.path.exists(testMask):
    os.makedirs(testMask)
    print("testMask 输出目录创建成功")

testImage 输出目录创建成功
testMask 输出目录创建成功


In [23]:
def file_name_path(file_dir, dir=True, file=False):
    """
    get root path,sub_dirs,all_sub_files
    :param file_dir:
    :return: dir or file
    """
    for root, dirs, files in os.walk(file_dir):
        if len(dirs) and dir:
            print("sub_dirs:", dirs)
            return dirs
        if len(files) and file:
            print("files:", files)
            return files

In [24]:
pathhgg_list = file_name_path(bratshgg_path)
pathlgg_list = file_name_path(bratslgg_path)

sub_dirs: ['Brats18_2013_10_1', 'Brats18_2013_11_1', 'Brats18_2013_12_1', 'Brats18_2013_13_1', 'Brats18_2013_14_1', 'Brats18_2013_17_1', 'Brats18_2013_18_1', 'Brats18_2013_19_1', 'Brats18_2013_20_1', 'Brats18_2013_21_1', 'Brats18_2013_22_1', 'Brats18_2013_23_1', 'Brats18_2013_25_1', 'Brats18_2013_26_1', 'Brats18_2013_27_1', 'Brats18_2013_2_1', 'Brats18_2013_3_1', 'Brats18_2013_4_1', 'Brats18_2013_5_1', 'Brats18_2013_7_1', 'Brats18_CBICA_AAB_1', 'Brats18_CBICA_AAG_1', 'Brats18_CBICA_AAL_1', 'Brats18_CBICA_AAP_1', 'Brats18_CBICA_ABB_1', 'Brats18_CBICA_ABE_1', 'Brats18_CBICA_ABM_1', 'Brats18_CBICA_ABN_1', 'Brats18_CBICA_ABO_1', 'Brats18_CBICA_ABY_1', 'Brats18_CBICA_ALN_1', 'Brats18_CBICA_ALU_1', 'Brats18_CBICA_ALX_1', 'Brats18_CBICA_AME_1', 'Brats18_CBICA_AMH_1', 'Brats18_CBICA_ANG_1', 'Brats18_CBICA_ANI_1', 'Brats18_CBICA_ANP_1', 'Brats18_CBICA_ANZ_1', 'Brats18_CBICA_AOD_1', 'Brats18_CBICA_AOH_1', 'Brats18_CBICA_AOO_1', 'Brats18_CBICA_AOP_1', 'Brats18_CBICA_AOZ_1', 'Brats18_CBICA_APR_1',

In [25]:
def normalize(slice, bottom=99, down=1):
    """
    normalize image with mean and std for regionnonzero,and clip the value into range
    :param slice:
    :param bottom:
    :param down:
    :return:
    """
    b = np.percentile(slice, bottom)
    t = np.percentile(slice, down)
    slice = np.clip(slice, t, b)

    image_nonzero = slice[np.nonzero(slice)]
    if np.std(slice) == 0 or np.std(image_nonzero) == 0:
        return slice
    else:
        tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
        # since the range of intensities is between 0 and 5000 ,
        # the min in the normalized slice corresponds to 0 intensity in unnormalized slice
        # the min is replaced with -9 just to keep track of 0 intensities
        # so that we can discard those intensities afterwards when sampling random patches
        tmp[tmp == tmp.min()] = -9
        return tmp

In [26]:
def crop_ceter(img,croph,cropw):   
    #for n_slice in range(img.shape[0]):
    height,width = img[0].shape 
    starth = height//2-(croph//2)
    startw = width//2-(cropw//2)        
    return img[:,starth:starth+croph,startw:startw+cropw]

#### 人工手動插入切片小實驗

In [43]:
a = np.array([[[1,1],[2,2]],[[3,3],[4,4]],[[5,5],[6,6]]])

In [44]:
b = np.zeros([2,2])

In [45]:
a.shape

(3, 2, 2)

In [46]:
b.shape

(2, 2)

In [47]:
a

array([[[1, 1],
        [2, 2]],

       [[3, 3],
        [4, 4]],

       [[5, 5],
        [6, 6]]])

In [48]:
c = np.insert(a,0,b,axis = 0)

In [49]:
c.shape[0]

4

In [50]:
c = np.insert(c,(c.shape[0]),b,axis = 0)

## BraTS2018 训练集

In [30]:
part = "hgg"
for subsetindex in range(len(pathhgg_list)):
    print(pathhgg_list[subsetindex])
    # 1、读取数据    
    brats_subset_path = bratshgg_path + "/" + str(pathhgg_list[subsetindex]) + "/"
    flair_image = brats_subset_path + str(pathhgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathhgg_list[subsetindex]) + mask_name
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    # 2、人工加入切片
    myblackslice = np.zeros([240,240])
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    
    # 3、对四个模态分别进行标准化 
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    
    # 4、裁剪
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    # 5、分块处理
    patch_block_size = BLOCKSIZE#BLOCKSIZE = (16, 160, 160) #每个分块的大小
    numberxy = patch_block_size[1]#160
    numberz = 8 #patch_block_size[0]
    width = np.shape(flair_crop)[1]#160
    height = np.shape(flair_crop)[2]#160
    imagez = np.shape(flair_crop)[0]#160
    block_width = np.array(patch_block_size)[1]#160
    block_height = np.array(patch_block_size)[2]#160
    blockz = np.array(patch_block_size)[0]#16
    stridewidth = (width - block_width) // numberxy#0
    strideheight = (height - block_height) // numberxy#0
    stridez = (imagez - blockz) // numberz#18
    print(stridez)
    step_width = width - (stridewidth * numberxy + block_width)#0
    step_width = step_width // 2#0
    print(step_width)
    step_height = height - (strideheight * numberxy + block_height)#0
    step_height = step_height // 2
    print(step_height)
    step_z = imagez - (stridez * numberz + blockz)
    step_z = step_z // 2
    print('step_z',step_z)#0
    hr_samples_flair_list = []
    hr_samples_t1_list = []
    hr_samples_t1ce_list = []
    hr_samples_t2_list = []
    hr_mask_samples_list = []
    patchnum = []
    for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
        for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
            for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
                if np.max(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height]) != 0:
                    print("切%d"%z)
                    patchnum.append(z)
                    hr_samples_flair_list.append(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_samples_t1_list.append(t1_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_samples_t1ce_list.append(t1ce_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_samples_t2_list.append(t2_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_mask_samples_list.append(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height])
    samples_flair = np.array(hr_samples_flair_list).reshape((len(hr_samples_flair_list), blockz, block_width, block_height))
    samples_t1 = np.array(hr_samples_t1_list).reshape((len(hr_samples_t1_list), blockz, block_width, block_height))
    samples_t1ce = np.array(hr_samples_t1ce_list).reshape((len(hr_samples_t1ce_list), blockz, block_width, block_height))
    samples_t2 = np.array(hr_samples_t2_list).reshape((len(hr_samples_t2_list), blockz, block_width, block_height))
    mask_samples = np.array(hr_mask_samples_list).reshape((len(hr_mask_samples_list), blockz, block_width, block_height))
    samples, imagez, height, width = np.shape(samples_flair)[0], np.shape(samples_flair)[1], \
                                     np.shape(samples_flair)[2], np.shape(samples_flair)[3]
    
    # 5、合并和保存
    for j in range(samples):
        print(samples)
        """
        merage 4 model image into 4 channel (imagez,width,height,channel)
        """
        fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
        filepath1 = trainImage + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        print(filepath1)
        filepath = trainMask + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        print(filepath)
        flairimage = samples_flair[j, :, :, :]
        flairimage = flairimage.astype(np.float)
        fourmodelimagearray[:, :, :, 0] = flairimage
        t1image = samples_t1[j, :, :, :]
        t1image = t1image.astype(np.float)
        fourmodelimagearray[:, :, :, 1] = t1image
        t1ceimage = samples_t1ce[j, :, :, :]
        t1ceimage = t1ceimage.astype(np.float)
        fourmodelimagearray[:, :, :, 2] = t1ceimage
        t2image = samples_t2[j, :, :, :]
        t2image = t2image.astype(np.float)
        fourmodelimagearray[:, :, :, 3] = t2image
        print(fourmodelimagearray.shape)
        np.save(filepath1, fourmodelimagearray)
        
        wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
        mask_one_sample = mask_samples[j, :, :, :]
        WT_Label = mask_one_sample.copy()
        WT_Label[mask_one_sample == 1] = 1.
        WT_Label[mask_one_sample == 2] = 1.
        WT_Label[mask_one_sample == 4] = 1.
        TC_Label = mask_one_sample.copy()
        TC_Label[mask_one_sample == 1] = 1.
        TC_Label[mask_one_sample == 2] = 0.
        TC_Label[mask_one_sample == 4] = 1.
        ET_Label = mask_one_sample.copy()
        ET_Label[mask_one_sample == 1] = 0.
        ET_Label[mask_one_sample == 2] = 0.
        ET_Label[mask_one_sample == 4] = 1.
        wt_tc_etMaskArray[:, :, :, 0] = WT_Label
        wt_tc_etMaskArray[:, :, :, 1] = TC_Label
        wt_tc_etMaskArray[:, :, :, 2] = ET_Label
        np.save(filepath, wt_tc_etMaskArray)
print("Done!")

Brats18_2013_10_1
18
0
0
step_z 0
切24
切32
切40
切48
切56
切64
切72
切80
切88
切96
切104
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_24.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_24.npy
(16, 160, 160, 4)


C:\Users\zhao'ming'zhang\AppData\Local\Temp\ipykernel_26452\250365200.py:114: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
C:\Users\zhao'ming'zhang\AppData\Local\Temp\ipykernel_26452\250365200.py:120: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  flairimage = flairimage

11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_32.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_32.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_40.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_40.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_48.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_48.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_56.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_56.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_64.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_64.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_72.npy
F:\Brats_data\process_data\trainMask\hgg_Brats18_2013_10_1_72.npy
(16, 160, 160, 4)
11
F:\Brats_data\process_data\trainImage\hgg_Brats18_2013_10_1_80.npy
F:\Bra

In [34]:
part = "lgg"
for subsetindex in range(len(pathlgg_list)):
    print(pathhgg_list[subsetindex])
    # 1、读取数据    
    brats_subset_path = bratslgg_path + "/" + str(pathlgg_list[subsetindex]) + "/"
    flair_image = brats_subset_path + str(pathlgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathlgg_list[subsetindex]) + mask_name
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    # 2、人工加入切片
    myblackslice = np.zeros([240,240])
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    # 3、对四个模态分别进行标准化 
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    # 4、裁剪
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    print(flair_crop.shape,t1_crop.shape,t1ce_crop.shape,t2_crop.shape)
    # 5、分块处理
    patch_block_size = BLOCKSIZE
    numberxy = patch_block_size[1]
    numberz = 8#patch_block_size[0]
    width = np.shape(flair_crop)[1]
    height = np.shape(flair_crop)[2]
    imagez = np.shape(flair_crop)[0]
    block_width = np.array(patch_block_size)[1]
    block_height = np.array(patch_block_size)[2]
    blockz = np.array(patch_block_size)[0]
    print('blockz',blockz)#32
    stridewidth = (width - block_width) // numberxy
    print('stridewidth',stridewidth)#0
    strideheight = (height - block_height) // numberxy
    print('strideheight',strideheight)#0
    stridez = (imagez - blockz) // numberz
    print('stridez',stridez)#18
    step_width = width - (stridewidth * numberxy + block_width)
    step_width = step_width // 2
    print('step_width',step_width)#0
    step_height = height - (strideheight * numberxy + block_height)
    step_height = step_height // 2
    print('step_height',step_height)#0
    step_z = imagez - (stridez * numberz + blockz)
    step_z = step_z // 2
    print('step_z',step_z)#0
    hr_samples_flair_list = []
    hr_samples_t1_list = []
    hr_samples_t1ce_list = []
    hr_samples_t2_list = []
    hr_mask_samples_list = []
    patchnum = []#  0          136                             8
    for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
        #                 0              160                                    160
        for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
            for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
                if np.max(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height]) != 0:
                    # print("切%d"%z)
                    # patchnum.append(z)
                    hr_samples_flair_list.append(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    print(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height].shape)
                    hr_samples_t1_list.append(t1_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_samples_t1ce_list.append(t1ce_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_samples_t2_list.append(t2_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                    hr_mask_samples_list.append(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height])
    print(len(hr_samples_flair_list))
    samples_flair = np.array(hr_samples_flair_list).reshape((len(hr_samples_flair_list), blockz, block_width, block_height))
    samples_t1 = np.array(hr_samples_t1_list).reshape((len(hr_samples_t1_list), blockz, block_width, block_height))
    samples_t1ce = np.array(hr_samples_t1ce_list).reshape((len(hr_samples_t1ce_list), blockz, block_width, block_height))
    samples_t2 = np.array(hr_samples_t2_list).reshape((len(hr_samples_t2_list), blockz, block_width, block_height))
    mask_samples = np.array(hr_mask_samples_list).reshape((len(hr_mask_samples_list), blockz, block_width, block_height))
    samples, imagez, height, width = np.shape(samples_flair)[0], np.shape(samples_flair)[1], \
                                     np.shape(samples_flair)[2], np.shape(samples_flair)[3]
    print(samples,imagez, height, width)
    # 5、合并和保存
    for j in range(samples):
        """
        merage 4 model image into 4 channel (imagez,width,height,channel)
        """
        fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
        filepath1 = trainImage + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        print(filepath1)
        filepath = trainMask + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        print(filepath)
        flairimage = samples_flair[j, :, :, :]
        flairimage = flairimage.astype(np.float)
        fourmodelimagearray[:, :, :, 0] = flairimage
        t1image = samples_t1[j, :, :, :]
        t1image = t1image.astype(np.float)
        fourmodelimagearray[:, :, :, 1] = t1image
        t1ceimage = samples_t1ce[j, :, :, :]
        t1ceimage = t1ceimage.astype(np.float)
        fourmodelimagearray[:, :, :, 2] = t1ceimage
        t2image = samples_t2[j, :, :, :]
        t2image = t2image.astype(np.float)
        fourmodelimagearray[:, :, :, 3] = t2image
        np.save(filepath1, fourmodelimagearray)
        
        wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
        mask_one_sample = mask_samples[j, :, :, :]
        WT_Label = mask_one_sample.copy()
        WT_Label[mask_one_sample == 1] = 1.
        WT_Label[mask_one_sample == 2] = 1.
        WT_Label[mask_one_sample == 4] = 1.
        TC_Label = mask_one_sample.copy()
        TC_Label[mask_one_sample == 1] = 1.
        TC_Label[mask_one_sample == 2] = 0.
        TC_Label[mask_one_sample == 4] = 1.
        ET_Label = mask_one_sample.copy()
        ET_Label[mask_one_sample == 1] = 0.
        ET_Label[mask_one_sample == 2] = 0.
        ET_Label[mask_one_sample == 4] = 1.
        wt_tc_etMaskArray[:, :, :, 0] = WT_Label
        wt_tc_etMaskArray[:, :, :, 1] = TC_Label
        wt_tc_etMaskArray[:, :, :, 2] = ET_Label
        np.save(filepath, wt_tc_etMaskArray)
print("Done!")

Brats18_2013_10_1
(160, 160, 160) (160, 160, 160) (160, 160, 160) (160, 160, 160)
blockz 16
stridewidth 0
strideheight 0
stridez 18
step_width 0
step_height 0
step_z 0
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
(16, 160, 160)
8
8 16 160 160


C:\Users\zhao'ming'zhang\AppData\Local\Temp\ipykernel_26452\4136219497.py:119: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)


IndexError: list index out of range

# 从BraTS2019多的数据集作为测试集

In [ ]:
import pandas as pd

In [ ]:
def compare_site_names(one,two):
    df1 = pd.read_csv(one, header=None)
    df2 = pd.read_csv(two, header=None)
    data1 = df1.to_dict(orient='list')[0]
    data2 = df2.to_dict(orient='list')[0]
    diff1 = set(data1).difference(data2)
    diff2 = set(data2).difference(data1)
    return list(diff2)

In [ ]:
hgg_diff = compare_site_names("18hgg.csv","19hgg.csv")

In [ ]:
len(hgg_diff)

49

In [ ]:
lgg_diff = compare_site_names("18lgg.csv","19lgg.csv")

In [ ]:
len(lgg_diff)

1

In [ ]:
pathhgg_list = []
pathlgg_list = []

In [ ]:
for idx in range(len(hgg_diff)):
    mystr = "BraTS19" + hgg_diff[idx]
    pathhgg_list.append(mystr)

In [ ]:
pathhgg_list

['BraTS19_CBICA_BDK_1',
 'BraTS19_CBICA_BHQ_1',
 'BraTS19_CBICA_AWV_1',
 'BraTS19_CBICA_BCL_1',
 'BraTS19_CBICA_BGE_1',
 'BraTS19_CBICA_BGX_1',
 'BraTS19_CBICA_AWX_1',
 'BraTS19_CBICA_BHV_1',
 'BraTS19_CBICA_AOC_1',
 'BraTS19_CBICA_BGW_1',
 'BraTS19_CBICA_BAN_1',
 'BraTS19_CBICA_AYC_1',
 'BraTS19_CBICA_BEM_1',
 'BraTS19_TMC_06290_1',
 'BraTS19_CBICA_BGN_1',
 'BraTS19_CBICA_AUW_1',
 'BraTS19_CBICA_BGG_1',
 'BraTS19_TMC_11964_1',
 'BraTS19_TMC_15477_1',
 'BraTS19_TMC_30014_1',
 'BraTS19_CBICA_BCF_1',
 'BraTS19_CBICA_ANV_1',
 'BraTS19_CBICA_BHZ_1',
 'BraTS19_CBICA_BKV_1',
 'BraTS19_CBICA_AYG_1',
 'BraTS19_TMC_06643_1',
 'BraTS19_CBICA_BIC_1',
 'BraTS19_CBICA_BGO_1',
 'BraTS19_CBICA_AVF_1',
 'BraTS19_CBICA_ASF_1',
 'BraTS19_CBICA_BGT_1',
 'BraTS19_CBICA_ASR_1',
 'BraTS19_CBICA_APK_1',
 'BraTS19_CBICA_BNR_1',
 'BraTS19_CBICA_BBG_1',
 'BraTS19_CBICA_AUA_1',
 'BraTS19_TMC_12866_1',
 'BraTS19_CBICA_BLJ_1',
 'BraTS19_CBICA_BAX_1',
 'BraTS19_CBICA_BAP_1',
 'BraTS19_CBICA_BGR_1',
 'BraTS19_CBICA_

In [ ]:
for idx in range(len(lgg_diff)):
    mystr = "BraTS19" + lgg_diff[idx]
    pathlgg_list.append(mystr)

In [ ]:
pathlgg_list

['BraTS19_TMC_09043_1']

In [ ]:
part = "hgg"
for subsetindex in range(len(pathhgg_list)):
    print(pathhgg_list[subsetindex])
    # 1、读取数据    
    brats_subset_path = bratshgg_path2 + "/" + str(pathhgg_list[subsetindex]) + "/"
    flair_image = brats_subset_path + str(pathhgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathhgg_list[subsetindex]) + mask_name
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    # 2、人工加入切片
    myblackslice = np.zeros([240,240])
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    # 3、对四个模态分别进行标准化 
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    # 4、裁剪
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    # 5、分块处理
    patch_block_size = BLOCKSIZE
    numberxy = patch_block_size[1]
    numberz = patch_block_size[0]
    width = np.shape(flair_crop)[1]
    height = np.shape(flair_crop)[2]
    imagez = np.shape(flair_crop)[0]
    block_width = np.array(patch_block_size)[1]
    block_height = np.array(patch_block_size)[2]
    blockz = np.array(patch_block_size)[0]
    stridewidth = (width - block_width) // numberxy
    strideheight = (height - block_height) // numberxy
    stridez = (imagez - blockz) // numberz
    step_width = width - (stridewidth * numberxy + block_width)
    step_width = step_width // 2
    step_height = height - (strideheight * numberxy + block_height)
    step_height = step_height // 2
    step_z = imagez - (stridez * numberz + blockz)
    step_z = step_z // 2
    hr_samples_flair_list = []
    hr_samples_t1_list = []
    hr_samples_t1ce_list = []
    hr_samples_t2_list = []
    hr_mask_samples_list = []
    patchnum = []
    for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
        for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
            for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
                #if np.max(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height]) != 0:
                print("切%d"%z)
                patchnum.append(z)
                hr_samples_flair_list.append(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t1_list.append(t1_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t1ce_list.append(t1ce_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t2_list.append(t2_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_mask_samples_list.append(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height])
    samples_flair = np.array(hr_samples_flair_list).reshape((len(hr_samples_flair_list), blockz, block_width, block_height))
    samples_t1 = np.array(hr_samples_t1_list).reshape((len(hr_samples_t1_list), blockz, block_width, block_height))
    samples_t1ce = np.array(hr_samples_t1ce_list).reshape((len(hr_samples_t1ce_list), blockz, block_width, block_height))
    samples_t2 = np.array(hr_samples_t2_list).reshape((len(hr_samples_t2_list), blockz, block_width, block_height))
    mask_samples = np.array(hr_mask_samples_list).reshape((len(hr_mask_samples_list), blockz, block_width, block_height))
    samples, imagez, height, width = np.shape(samples_flair)[0], np.shape(samples_flair)[1], \
                                     np.shape(samples_flair)[2], np.shape(samples_flair)[3]
    # 5、合并和保存
    for j in range(samples):
        """
        merage 4 model image into 4 channel (imagez,width,height,channel)
        """
        fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
        filepath1 = testImage + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        filepath = testMask + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        flairimage = samples_flair[j, :, :, :]
        flairimage = flairimage.astype(np.float)
        fourmodelimagearray[:, :, :, 0] = flairimage
        t1image = samples_t1[j, :, :, :]
        t1image = t1image.astype(np.float)
        fourmodelimagearray[:, :, :, 1] = t1image
        t1ceimage = samples_t1ce[j, :, :, :]
        t1ceimage = t1ceimage.astype(np.float)
        fourmodelimagearray[:, :, :, 2] = t1ceimage
        t2image = samples_t2[j, :, :, :]
        t2image = t2image.astype(np.float)
        fourmodelimagearray[:, :, :, 3] = t2image
        np.save(filepath1, fourmodelimagearray)
        
        wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
        mask_one_sample = mask_samples[j, :, :, :]
        WT_Label = mask_one_sample.copy()
        WT_Label[mask_one_sample == 1] = 1.
        WT_Label[mask_one_sample == 2] = 1.
        WT_Label[mask_one_sample == 4] = 1.
        TC_Label = mask_one_sample.copy()
        TC_Label[mask_one_sample == 1] = 1.
        TC_Label[mask_one_sample == 2] = 0.
        TC_Label[mask_one_sample == 4] = 1.
        ET_Label = mask_one_sample.copy()
        ET_Label[mask_one_sample == 1] = 0.
        ET_Label[mask_one_sample == 2] = 0.
        ET_Label[mask_one_sample == 4] = 1.
        wt_tc_etMaskArray[:, :, :, 0] = WT_Label
        wt_tc_etMaskArray[:, :, :, 1] = TC_Label
        wt_tc_etMaskArray[:, :, :, 2] = ET_Label
        np.save(filepath, wt_tc_etMaskArray)
print("Done!")

BraTS19_CBICA_BDK_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BHQ_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AWV_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BCL_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BGE_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BGX_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AWX_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BHV_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AOC_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BGW_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BAN_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AYC_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BEM_1
切0
切32
切64
切96
切128
BraTS19_TMC_06290_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BGN_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AUW_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BGG_1
切0
切32
切64
切96
切128
BraTS19_TMC_11964_1
切0
切32
切64
切96
切128
BraTS19_TMC_15477_1
切0
切32
切64
切96
切128
BraTS19_TMC_30014_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BCF_1
切0
切32
切64
切96
切128
BraTS19_CBICA_ANV_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BHZ_1
切0
切32
切64
切96
切128
BraTS19_CBICA_BKV_1
切0
切32
切64
切96
切128
BraTS19_CBICA_AYG_1
切0
切32
切64
切96
切128


In [ ]:
pathlgg_list

['BraTS19_TMC_09043_1']

In [ ]:
part = "lgg"
for subsetindex in range(len(pathlgg_list)):
    print(pathlgg_list[subsetindex])
    # 1、读取数据    
    brats_subset_path = bratslgg_path2 + "/" + str(pathlgg_list[subsetindex]) + "/"
    flair_image = brats_subset_path + str(pathlgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathlgg_list[subsetindex]) + mask_name
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    # 2、人工加入切片
    myblackslice = np.zeros([240,240])
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,0,myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    flair_array = np.insert(flair_array,flair_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,0,myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1_array = np.insert(t1_array,t1_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,0,myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t1ce_array = np.insert(t1ce_array,t1ce_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,0,myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    t2_array = np.insert(t2_array,t2_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,0,myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    mask_array = np.insert(mask_array,mask_array.shape[0],myblackslice,axis = 0)
    # 3、对四个模态分别进行标准化 
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    # 4、裁剪
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    # 5、分块处理
    patch_block_size = BLOCKSIZE
    numberxy = patch_block_size[1]
    numberz = patch_block_size[0]
    width = np.shape(flair_crop)[1]
    height = np.shape(flair_crop)[2]
    imagez = np.shape(flair_crop)[0]
    block_width = np.array(patch_block_size)[1]
    block_height = np.array(patch_block_size)[2]
    blockz = np.array(patch_block_size)[0]
    stridewidth = (width - block_width) // numberxy
    strideheight = (height - block_height) // numberxy
    stridez = (imagez - blockz) // numberz
    step_width = width - (stridewidth * numberxy + block_width)
    step_width = step_width // 2
    step_height = height - (strideheight * numberxy + block_height)
    step_height = step_height // 2
    step_z = imagez - (stridez * numberz + blockz)
    step_z = step_z // 2
    hr_samples_flair_list = []
    hr_samples_t1_list = []
    hr_samples_t1ce_list = []
    hr_samples_t2_list = []
    hr_mask_samples_list = []
    patchnum = []
    for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
        for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
            for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
                #if np.max(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height]) != 0:
                print("切%d"%z)
                patchnum.append(z)
                hr_samples_flair_list.append(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t1_list.append(t1_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t1ce_list.append(t1ce_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_samples_t2_list.append(t2_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_mask_samples_list.append(mask_crop[z:z + blockz, x:x + block_width, y:y + block_height])
    samples_flair = np.array(hr_samples_flair_list).reshape((len(hr_samples_flair_list), blockz, block_width, block_height))
    samples_t1 = np.array(hr_samples_t1_list).reshape((len(hr_samples_t1_list), blockz, block_width, block_height))
    samples_t1ce = np.array(hr_samples_t1ce_list).reshape((len(hr_samples_t1ce_list), blockz, block_width, block_height))
    samples_t2 = np.array(hr_samples_t2_list).reshape((len(hr_samples_t2_list), blockz, block_width, block_height))
    mask_samples = np.array(hr_mask_samples_list).reshape((len(hr_mask_samples_list), blockz, block_width, block_height))
    samples, imagez, height, width = np.shape(samples_flair)[0], np.shape(samples_flair)[1], \
                                     np.shape(samples_flair)[2], np.shape(samples_flair)[3]
    # 5、合并和保存
    for j in range(samples):
        """
        merage 4 model image into 4 channel (imagez,width,height,channel)
        """
        fourmodelimagearray = np.zeros((imagez, height, width, 4), np.float)
        filepath1 = testImage + "\\" + part + "_" + pathlgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        filepath = testMask + "\\" + part + "_" + pathlgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
        flairimage = samples_flair[j, :, :, :]
        flairimage = flairimage.astype(np.float)
        fourmodelimagearray[:, :, :, 0] = flairimage
        t1image = samples_t1[j, :, :, :]
        t1image = t1image.astype(np.float)
        fourmodelimagearray[:, :, :, 1] = t1image
        t1ceimage = samples_t1ce[j, :, :, :]
        t1ceimage = t1ceimage.astype(np.float)
        fourmodelimagearray[:, :, :, 2] = t1ceimage
        t2image = samples_t2[j, :, :, :]
        t2image = t2image.astype(np.float)
        fourmodelimagearray[:, :, :, 3] = t2image
        np.save(filepath1, fourmodelimagearray)
        
        wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
        mask_one_sample = mask_samples[j, :, :, :]
        WT_Label = mask_one_sample.copy()
        WT_Label[mask_one_sample == 1] = 1.
        WT_Label[mask_one_sample == 2] = 1.
        WT_Label[mask_one_sample == 4] = 1.
        TC_Label = mask_one_sample.copy()
        TC_Label[mask_one_sample == 1] = 1.
        TC_Label[mask_one_sample == 2] = 0.
        TC_Label[mask_one_sample == 4] = 1.
        ET_Label = mask_one_sample.copy()
        ET_Label[mask_one_sample == 1] = 0.
        ET_Label[mask_one_sample == 2] = 0.
        ET_Label[mask_one_sample == 4] = 1.
        wt_tc_etMaskArray[:, :, :, 0] = WT_Label
        wt_tc_etMaskArray[:, :, :, 1] = TC_Label
        wt_tc_etMaskArray[:, :, :, 2] = ET_Label
        np.save(filepath, wt_tc_etMaskArray)
print("Done!")

BraTS19_TMC_09043_1
切0
切32
切64
切96
切128
Done!
